In [1]:
!pip3 install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!mkdir ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c g-research-crypto-forecasting

100% 1.13G/1.13G [00:07<00:00, 198MB/s]
100% 1.13G/1.13G [00:07<00:00, 167MB/s]


In [9]:
!unzip /content/g-research-crypto-forecasting.zip

Archive:  /content/g-research-crypto-forecasting.zip
  inflating: asset_details.csv       
  inflating: example_sample_submission.csv  
  inflating: example_test.csv        
  inflating: gresearch_crypto/__init__.py  
  inflating: gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: supplemental_train.csv  
  inflating: train.csv               


In [14]:
!mkdir output

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
data = pd.read_csv('/content/train.csv')

In [12]:
data.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [13]:
assets = pd.read_csv('/content/asset_details.csv')

In [16]:
assets

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


In [17]:
coins = ['Bitcoin', 'Ethereum', 'TRON', 'IOTA', 'Dogecoin']

In [21]:
coins_info = assets.loc[assets['Asset_Name'].isin(coins)]
list(coins_info['Asset_ID'])

[1, 6, 13, 8, 4]

In [26]:
coins = data.loc[data['Asset_ID'].isin(list(coins_info['Asset_ID']))]

In [32]:
coins.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
2,1514764860,1,229.0,13835.1940,14013.80,13666.11,13850.1760,31.550062,13827.062093,-0.014643
5,1514764860,6,173.0,738.3025,746.00,732.51,738.5075,335.987856,738.839291,-0.004809
10,1514764920,1,235.0,13835.0360,14052.30,13680.00,13828.1020,31.046432,13840.362591,-0.015037
13,1514764920,6,192.0,738.5075,745.14,732.49,738.2600,232.793141,738.268967,-0.004441
18,1514764980,1,528.0,13823.9000,14000.40,13601.00,13801.3140,55.061820,13806.068014,-0.010309


In [35]:
coins_info.reset_index(inplace=True)
coins_info

,index,Asset_ID,Weight,Asset_Name
0,2,1,6.779922,Bitcoin
1,5,6,5.894403,Ethereum
2,8,13,1.791759,TRON
3,11,8,1.098612,IOTA
4,13,4,3.555348,Dogecoin


In [37]:
coins_info.drop(columns = ['index'], inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
coins_info

,Asset_ID,Weight,Asset_Name
0,1,6.779922,Bitcoin
1,6,5.894403,Ethereum
2,13,1.791759,TRON
3,8,1.098612,IOTA
4,4,3.555348,Dogecoin


In [42]:
coins_info['Asset_ID'][1], coins_info['Asset_Name'][0]

(6, 'Bitcoin')

In [43]:
coins_dict = {}
for i in range(len(coins_info)):
  coins_dict[coins_info['Asset_ID'][i]] = coins_info['Asset_Name'][i]

In [44]:
coins_dict

{1: 'Bitcoin', 4: 'Dogecoin', 6: 'Ethereum', 8: 'IOTA', 13: 'TRON'}

In [50]:
coins.rename(columns = {'Asset_ID':'Asset'}, inplace = True)
coins.replace({'Asset' : coins_dict}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [54]:
coins.to_csv('/content/output/data.csv')

In [55]:
train = pd.read_csv('/content/output/data.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8535979 entries, 0 to 8535978
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   timestamp   int64  
 2   Asset       object 
 3   Count       float64
 4   Open        float64
 5   High        float64
 6   Low         float64
 7   Close       float64
 8   Volume      float64
 9   VWAP        float64
 10  Target      float64
dtypes: float64(8), int64(2), object(1)
memory usage: 716.4+ MB


In [58]:
train.drop(columns = ['Unnamed: 0'], inplace=True)

In [59]:
train

,timestamp,Asset,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,Bitcoin,229.0,13835.194000,14013.800000,13666.11000,13850.176000,3.155006e+01,13827.062093,-0.014643
1,1514764860,Ethereum,173.0,738.302500,746.000000,732.51000,738.507500,3.359879e+02,738.839291,-0.004809
2,1514764920,Bitcoin,235.0,13835.036000,14052.300000,13680.00000,13828.102000,3.104643e+01,13840.362591,-0.015037
3,1514764920,Ethereum,192.0,738.507500,745.140000,732.49000,738.260000,2.327931e+02,738.268967,-0.004441
4,1514764980,Bitcoin,528.0,13823.900000,14000.400000,13601.00000,13801.314000,5.506182e+01,13806.068014,-0.010309
...,...,...,...,...,...,...,...,...,...,...
8535974,1632182400,Bitcoin,2698.0,43009.961250,43048.510000,42961.64000,43002.505000,1.282068e+02,43011.414052,NaN
8535975,1632182400,Dogecoin,537.0,0.208622,0.208800,0.20820,0.208294,1.140946e+06,0.208515,NaN
8535976,1632182400,Ethereum,2205.0,2976.858333,2978.820000,2969.35000,2972.603333,1.204826e+03,2975.213919,NaN
8535977,1632182400,IOTA,82.0,1.284250,1.303100,1.26520,1.282550,8.542407e+03,1.284479,NaN


In [63]:
coins_info

,Asset_ID,Weight,Asset_Name
0,1,6.779922,Bitcoin
1,6,5.894403,Ethereum
2,13,1.791759,TRON
3,8,1.098612,IOTA
4,4,3.555348,Dogecoin


In [65]:
bitcoin = train.loc[train['Asset'].isin(['Bitcoin'])]
ethereum = train.loc[train['Asset'].isin(['Ethereum'])]
tron = train.loc[train['Asset'].isin(['TRON'])]
iota = train.loc[train['Asset'].isin(['IOTA'])]
dogecoin = train.loc[train['Asset'].isin(['Dogecoin'])]

In [66]:
!mkdir coins

In [67]:
bitcoin.to_csv('/content/coins/bitcoin.csv')
ethereum.to_csv('/content/coins/ethereum.csv')
tron.to_csv('/content/coins/tron.csv')
iota.to_csv('/content/coins/iota.csv')
dogecoin.to_csv('/content/coins/dogecoin.csv')